# Import the libraries that are necesaries

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import getpass  # To get the password without showing the input
from sqlalchemy import create_engine, text
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import nltk
from nltk.corpus import stopwords
from pycaret.regression import *

Connect to MySQL

In [2]:
password = getpass.getpass()
user = 'root'
database_name = 'steam_games'
connection_string = f'mysql+pymysql://{user}:' + password + '@localhost/' + database_name
engine = create_engine(connection_string)

### Read the CSVs

In [3]:
# # Specify the directory containing the CSV files
# directory_path = '../data/clean/'

# # Specify the order of CSV files
# csv_order = ['game.csv', 'languages.csv', 'platform.csv', 'metacritic.csv', 'user_feedback.csv',
#              'playtime.csv', 'development.csv', 'categorization.csv', 'media.csv']

# # Initialize an empty DataFrame to store the merged result
# game_data = pd.DataFrame()

# # Loop through each CSV file, read it into a DataFrame, and merge it with the existing DataFrame
# for csv_file in csv_order:
#     file_path = os.path.join(directory_path, csv_file)
#     df = pd.read_csv(file_path)

#     # Merge based on the 'appid' column
#     if game_data.empty:
#         game_data = df
#     else:
#         game_data = pd.merge(game_data, df, on='appid', how='outer')

Normalize the dataset

In [4]:
# # Download NLTK stopwords data
# nltk.download('stopwords')

# # Columns to convert to lists
# columns_to_convert = ['supported_languages', 'full_audio_languages', 'categories', 'genres', 'tags']

# # Create a copy of the original DataFrame
# df_normalized = game_data.copy()

# # Fill NaN values with an empty string
# df_normalized[columns_to_convert] = df_normalized[columns_to_convert].fillna('')

# # Normalize numerical features using StandardScaler
# numerical_columns = df_normalized.select_dtypes(include=['float64', 'int64']).columns.difference(['appid'])
# scaler = StandardScaler()
# df_normalized[numerical_columns] = scaler.fit_transform(df_normalized[numerical_columns])

# # One-hot encode categorical features
# categorical_columns = df_normalized.select_dtypes(include=['bool']).columns.difference(['appid'])
# df_normalized = pd.get_dummies(df_normalized, columns=categorical_columns, drop_first=True)

# # Use CountVectorizer to convert text columns to bag-of-words representation
# count_vectorizer = CountVectorizer(stop_words=stopwords.words('english'))  # Use English stopwords
# for column in columns_to_convert:
#     column_bow = count_vectorizer.fit_transform(df_normalized[column])
#     df_normalized = pd.concat([df_normalized, pd.DataFrame(column_bow.toarray(), columns=count_vectorizer.get_feature_names_out([column]))], axis=1)
#     df_normalized.drop(column, axis=1, inplace=True)

# # Convert 'appid' column to integers
# df_normalized['appid'] = df_normalized['appid'].astype(int)

# df_normalized

# 1. Game Popularity vs. Features:
- *Theory*: Peak concurrent users (peak_ccu) of games are generally higher when their estimated owners are higher.
- *Analysis*: Peak_ccu and estimated owners' correlation with one another.

In [5]:
with engine.connect() as connection:
    # Query to extract relevant columns
    txt = '''SELECT
                appid,
                estimated_owners,
                peak_ccu
            FROM
                game;'''
    query = text(txt)
    result = connection.execute(query)
    game_df = pd.DataFrame(result.all())

game_df

,appid,estimated_owners,peak_ccu
0,10,15000000.0,13230.0
1,20,7500000.0,110.0
2,30,7500000.0,106.0
3,40,7500000.0,2.0
4,50,7500000.0,120.0
...,...,...,...
85098,2760980,0.0,0.0
85099,2761170,0.0,0.0
85100,2763480,0.0,0.0
85101,2764930,0.0,0.0


In [6]:
# Fill NaN values with an empty string (if needed)
game_df = game_df.fillna('')

# Normalize numerical features using StandardScaler
numerical_columns = game_df.select_dtypes(include=['float64', 'int64']).columns.difference(['appid'])
scaler = StandardScaler()
game_df[numerical_columns] = scaler.fit_transform(game_df[numerical_columns])

In [7]:
# Initialize PyCaret
reg_setup = setup(
    data=game_df,
    target='estimated_owners',
    train_size=0.8,
    session_id=42,
    numeric_features=['peak_ccu']
)

# Compare different regression models
reg_models = compare_models()

,Description,Value
0,Session id,42
1,Target,estimated_owners
2,Target type,Regression
3,Original data shape,"(85103, 3)"
4,Transformed data shape,"(85103, 3)"
5,Transformed train set shape,"(68082, 3)"
6,Transformed test set shape,"(17021, 3)"
7,Numeric features,1
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
gbr,Gradient Boosting Regressor,0.0689,0.5959,0.7198,0.3773,0.1074,0.7444,0.5280
rf,Random Forest Regressor,0.0713,0.6321,0.7415,0.3655,0.1173,0.8173,1.6090
lightgbm,Light Gradient Boosting Machine,0.0733,0.6562,0.7570,0.3182,0.1213,0.7655,0.1430
et,Extra Trees Regressor,0.0756,0.7122,0.7850,0.2734,0.1233,0.8494,0.6690
lr,Linear Regression,0.1283,0.6959,0.7897,0.2316,0.1418,1.4256,0.4120
ridge,Ridge Regression,0.1283,0.6959,0.7897,0.2316,0.1418,1.4257,0.0130
lar,Least Angle Regression,0.1283,0.6959,0.7897,0.2316,0.1418,1.4256,0.0120
br,Bayesian Ridge,0.1283,0.6959,0.7897,0.2316,0.1418,1.4257,0.0130
en,Elastic Net,0.1418,1.0936,0.9462,0.0448,0.1553,1.5944,0.0140
lasso,Lasso Regression,0.1441,1.1215,0.9609,0.0120,0.1618,1.6247,0.2690


In [8]:
# Create a Gradient Boosting Regressor model
gbm_model = create_model('gbr')

# Tune the Gradient Boosting Regressor model
tuned_gbm_model = tune_model(gbm_model)

# Evaluate the tuned model
evaluate_model(tuned_gbm_model)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0705,0.8332,0.9128,0.7978,0.1032,0.7908
1,0.0574,0.3353,0.5791,0.4870,0.0892,0.7411
2,0.0581,0.2083,0.4564,0.4736,0.1010,0.6996
3,0.0858,1.1834,1.0879,0.0098,0.1222,0.8253
4,0.0799,0.9422,0.9707,0.5032,0.1100,0.6737
5,0.0720,0.3728,0.6106,0.2809,0.1137,0.7545
6,0.0632,0.3847,0.6202,0.1667,0.1044,0.7244
7,0.0591,0.1620,0.4025,0.5665,0.1036,0.7294
8,0.0830,1.3958,1.1815,0.0203,0.1160,0.7332


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0816,1.6141,1.2705,0.6083,0.1139,0.8917
1,0.0673,0.3908,0.6252,0.4021,0.1091,0.8664
2,0.0643,0.1913,0.4373,0.5166,0.1089,0.7888
3,0.0879,0.8138,0.9021,0.3191,0.1327,0.8641
4,0.0826,0.8599,0.9273,0.5466,0.1201,0.7209
5,0.0761,0.3974,0.6304,0.2336,0.1215,0.7487
6,0.0706,0.4347,0.6593,0.0585,0.1222,0.8481
7,0.0672,0.1747,0.4180,0.5325,0.1214,0.8392
8,0.0723,0.3347,0.5785,0.7651,0.1147,0.8458


Fitting 10 folds for each of 10 candidates, totalling 100 fits


interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [9]:
# # Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(game_df[['estimated_owners']], game_df['peak_ccu'], test_size=0.2, random_state=42)

# # Fit a linear regression model on the training set
# model = LinearRegression().fit(X_train, y_train)

# # Predict on the test set
# y_pred = model.predict(X_test)

# # Evaluate the model
# mse = mean_squared_error(y_test, y_pred)
# r2 = r2_score(y_test, y_pred)

# print(f'Mean Squared Error: {mse}')
# print(f'R-squared: {r2}')

# # Print the coefficients
# print(f'Intercept: {model.intercept_}, Coefficient: {model.coef_}')

# Scatter plot of actual vs predicted values
# sns.scatterplot(x=X_test['estimated_owners'], y=y_test, label='Actual')
# sns.scatterplot(x=X_test['estimated_owners'], y=y_pred, label='Predicted')
# plt.title('Actual vs Predicted Peak Concurrent Users')
# plt.xlabel('Estimated Owners')
# plt.ylabel('Peak Concurrent Users')
# plt.legend()
# plt.show()

# Regression plot
# sns.regplot(x=X_test['estimated_owners'], y=y_test, scatter_kws={'alpha':0.3}, label='Actual')
# sns.regplot(x=X_test['estimated_owners'], y=y_pred, scatter_kws={'alpha':0}, line_kws={'color':'red'}, label='Regression Line')
# plt.title('Regression Plot for Peak Concurrent Users')
# plt.xlabel('Estimated Owners')
# plt.ylabel('Peak Concurrent Users')
# plt.legend()
# plt.show()

# 2. Impact of Platforms on Game Adoption:
- *Theory*: There are more players in games that are accessible on several platforms (Windows, Mac, and Linux).
- *Analysis*: Examine the approximate ownership of games that are accessible across several platforms.

In [10]:
with engine.connect() as connection:
    # Query to extract relevant columns
    txt = '''SELECT 
                game.estimated_owners,
                game.peak_ccu, 
                platform.windows, 
                platform.mac, 
                platform.linux
            FROM game
            JOIN platform ON game.appid = platform.appid;'''
    query = text(txt)
    result = connection.execute(query)
    platform_df = pd.DataFrame(result.all())

platform_df

,estimated_owners,peak_ccu,windows,mac,linux
0,15000000.0,13230.0,1,1,1
1,7500000.0,110.0,1,1,1
2,7500000.0,106.0,1,1,1
3,7500000.0,2.0,1,1,1
4,7500000.0,120.0,1,1,1
...,...,...,...,...,...
85098,0.0,0.0,1,0,0
85099,0.0,0.0,1,0,0
85100,0.0,0.0,1,0,0
85101,0.0,0.0,1,0,0


In [11]:
# Normalize numerical features using StandardScaler
numerical_columns = platform_df.select_dtypes(include=['float64', 'int64']).columns.difference(['appid'])
scaler = StandardScaler()
platform_df[numerical_columns] = scaler.fit_transform(platform_df[numerical_columns])

# Concatenate 'windows', 'mac', and 'linux' to create a single feature 'platform'
platform_df['platform'] = platform_df['windows'] + platform_df['mac'] + platform_df['linux']

In [12]:
# Initialize PyCaret
reg_setup = setup(
    data=platform_df,
    target='estimated_owners',
    session_id=42,
    train_size=0.8,
    numeric_features=['peak_ccu']
)

# Compare different regression models
reg_models = compare_models()

,Description,Value
0,Session id,42
1,Target,estimated_owners
2,Target type,Regression
3,Original data shape,"(85103, 6)"
4,Transformed data shape,"(85103, 6)"
5,Transformed train set shape,"(68082, 6)"
6,Transformed test set shape,"(17021, 6)"
7,Numeric features,1
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lr,Linear Regression,0.1132,0.7016,0.7935,0.2226,0.1483,1.0036,0.0280
ridge,Ridge Regression,0.1132,0.7016,0.7935,0.2226,0.1484,1.0019,0.0250
lar,Least Angle Regression,0.1132,0.7016,0.7935,0.2226,0.1484,1.0019,0.0250
br,Bayesian Ridge,0.1132,0.7016,0.7935,0.2226,0.1484,1.0019,0.0260
omp,Orthogonal Matching Pursuit,0.1144,0.7032,0.7944,0.2206,0.1568,0.8871,0.0280
knn,K Neighbors Regressor,0.0912,0.8723,0.8635,0.1396,0.1475,1.0437,0.5050
lightgbm,Light Gradient Boosting Machine,0.0897,0.8414,0.8546,0.1340,0.1415,1.0223,0.1540
huber,Huber Regressor,0.0764,1.0129,0.9167,0.0936,0.1524,0.4778,0.5090
rf,Random Forest Regressor,0.0938,0.9317,0.8962,0.0582,0.1509,0.9692,0.2930
gbr,Gradient Boosting Regressor,0.0911,1.0288,0.9230,0.0446,0.1355,1.0027,0.2650


In [13]:
# Create a Linear Regression model
lr_model = create_model('lr')

# Evaluate the model
evaluate_model(lr_model)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.1170,1.1443,1.0697,0.7223,0.1420,0.9956
1,0.1109,0.5802,0.7617,0.1123,0.1485,1.0622
2,0.1038,0.3261,0.5711,0.1758,0.1412,0.9845
3,0.1136,0.5167,0.7189,0.5676,0.1493,1.0227
4,0.1313,1.4032,1.1846,0.2601,0.1628,0.9483
5,0.1108,0.4044,0.6359,0.2200,0.1508,1.0070
6,0.1033,0.4295,0.6554,0.0697,0.1398,1.0108
7,0.1067,0.3269,0.5718,0.1254,0.1480,0.9951
8,0.1304,1.6552,1.2866,-0.1618,0.1544,0.9762


interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [14]:
# # Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(platform_df[['platform']], platform_df['estimated_owners'], test_size=0.2, random_state=42)

# # Fit a linear regression model
# linear_model = LinearRegression()
# linear_model.fit(X_train, y_train)

# # Make predictions on the test set
# y_pred = linear_model.predict(X_test)

# # Evaluate the model
# mse = mean_squared_error(y_test, y_pred)
# r2 = r2_score(y_test, y_pred)

# print(f'Mean Squared Error: {mse}')
# print(f'R-squared: {r2}')

# # Print the coefficients
# print(f'Intercept: {linear_model.intercept_}, Coefficient: {linear_model.coef_}')

# # Scatter plot of actual vs predicted values
# plt.scatter(X_test['platform'], y_test, label='Actual')
# plt.scatter(X_test['platform'], y_pred, label='Predicted')
# plt.title('Actual vs Predicted Estimated Owners')
# plt.xlabel('Platform')
# plt.ylabel('Estimated Owners')
# plt.legend()
# plt.show()

# # Regression plot
# sns.regplot(x=X_test['platform'], y=y_test, scatter_kws={'alpha': 0.3}, label='Actual')
# sns.regplot(x=X_test['platform'], y=y_pred, scatter_kws={'alpha': 0}, line_kws={'color': 'red'}, label='Regression Line')
# plt.title('Regression Plot for Estimated Owners by Combined Platform')
# plt.xlabel('Combined Platform')
# plt.ylabel('Estimated Owners')
# plt.legend()
# plt.show()

# 3. Metacritic Score and User Feedback:
- *Theory*: The user and Metacritic scores are positively correlated.
- *Analysis*: Comparison of the user_score and metacritic_score correlation.

In [15]:
with engine.connect() as connection:
    # Query to extract relevant columns
    txt = '''SELECT
                game.appid,
                game.estimated_owners,
                metacritic.metacritic_score,
                user_feedback.user_score
            FROM game
            JOIN metacritic ON game.appid = metacritic.appid
            JOIN user_feedback ON game.appid = user_feedback.appid;'''
    query = text(txt)
    result = connection.execute(query)
    user_score_df = pd.DataFrame(result.all())

user_score_df

,appid,estimated_owners,metacritic_score,user_score
0,10,15000000.0,88.0,0.0
1,20,7500000.0,0.0,0.0
2,30,7500000.0,79.0,0.0
3,40,7500000.0,0.0,0.0
4,50,7500000.0,0.0,0.0
...,...,...,...,...
85098,2760980,0.0,0.0,0.0
85099,2761170,0.0,0.0,0.0
85100,2763480,0.0,0.0,0.0
85101,2764930,0.0,0.0,0.0


In [16]:
# Fill NaN values (if needed)
user_score_df = user_score_df.fillna('')

# Normalize numerical features using StandardScaler
numerical_columns = user_score_df.select_dtypes(include=['float64', 'int64']).columns.difference(['appid'])
scaler = StandardScaler()
user_score_df[numerical_columns] = scaler.fit_transform(user_score_df[numerical_columns])

In [17]:
# Initialize PyCaret for regression
reg_setup = setup(
    data=user_score_df,
    target='estimated_owners',
    session_id=42,
    numeric_features=['metacritic_score', 'user_score']
)

# Compare different regression models
reg_models = compare_models()

,Description,Value
0,Session id,42
1,Target,estimated_owners
2,Target type,Regression
3,Original data shape,"(85103, 4)"
4,Transformed data shape,"(85103, 4)"
5,Transformed train set shape,"(59572, 4)"
6,Transformed test set shape,"(25531, 4)"
7,Numeric features,2
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lr,Linear Regression,0.1173,0.7836,0.8390,0.0505,0.1671,1.4627,0.0170
lar,Least Angle Regression,0.1173,0.7836,0.8390,0.0505,0.1671,1.4627,0.0140
br,Bayesian Ridge,0.1173,0.7836,0.8390,0.0505,0.1670,1.4627,0.0170
ridge,Ridge Regression,0.1173,0.7836,0.8390,0.0505,0.1671,1.4627,0.0150
huber,Huber Regressor,0.0874,0.8084,0.8531,0.0154,0.1648,0.5497,0.0820
llar,Lasso Least Angle Regression,0.1400,0.8085,0.8536,0.0128,0.1616,1.5688,0.0160
omp,Orthogonal Matching Pursuit,0.1400,0.8085,0.8536,0.0128,0.1616,1.5688,0.0160
en,Elastic Net,0.1400,0.8085,0.8536,0.0128,0.1616,1.5688,0.0140
lasso,Lasso Regression,0.1400,0.8085,0.8536,0.0128,0.1616,1.5688,0.0130
dummy,Dummy Regressor,0.1289,0.8171,0.8586,-0.0004,0.1835,1.0028,0.0180


In [18]:
# Create a Linear Regression model
lr_model = create_model('lr')

# Evaluate the model
evaluate_model(lr_model)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.1116,0.6846,0.8274,0.0490,0.1573,1.4537
1,0.1115,0.4212,0.6490,0.0369,0.1653,1.4822
2,0.1248,1.3078,1.1436,0.0302,0.1744,1.3210
3,0.1425,1.8817,1.3717,0.0337,0.1911,1.3017
4,0.1212,0.6719,0.8197,0.0501,0.1766,1.5623
5,0.1028,0.1744,0.4176,0.1123,0.1463,1.6246
6,0.1101,0.7003,0.8368,0.0348,0.1598,1.3640
7,0.1234,1.2176,1.1034,0.0421,0.1698,1.5000
8,0.1157,0.5397,0.7346,0.0314,0.1697,1.5472


interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [19]:
# # Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(df_normalized[['estimated_owners', 'metacritic_score', 'user_score']], df_normalized['peak_ccu'], test_size=0.2, random_state=42)

# # Fit a linear regression model
# model_rgr = LinearRegression().fit(X_train, y_train)

# # Make predictions on the test set
# y_pred = model_rgr.predict(X_test)

# # Evaluate the model
# mse = mean_squared_error(y_test, y_pred)
# r2 = r2_score(y_test, y_pred)

# print(f'Mean Squared Error: {mse}')
# print(f'R-squared: {r2}')

# # Print the coefficients
# print(f'Intercept: {model_rgr.intercept_}, Coefficients: {model_rgr.coef_}')

# # Scatter plot of actual vs predicted values
# plt.scatter(X_test['estimated_owners'], y_test, label='Actual')
# plt.scatter(X_test['estimated_owners'], y_pred, label='Predicted')
# plt.title('Actual vs Predicted Estimated Owners')
# plt.xlabel('Estimated Owners')
# plt.ylabel('User Score')
# plt.legend()
# plt.show()

# # Regression plot
# sns.regplot(x=X_test['estimated_owners'], y=y_test, scatter_kws={'alpha': 0.3}, label='Actual')
# sns.regplot(x=X_test['estimated_owners'], y=y_pred, scatter_kws={'alpha': 0}, line_kws={'color': 'red'}, label='Regression Line')
# plt.title('Regression Plot for Estimated Owners vs User Score')
# plt.xlabel('Estimated Owners')
# plt.ylabel('User Score')
# plt.legend()
# plt.show()

# 4. Effect of Game Features on Reviews:
- *Theory*: There are more achievements in games that have received favorable reviews.
- *Analysis*: Examine the typical quantity of achievements in games that have both favorable and unfavorable evaluations.

In [20]:
with engine.connect() as connection:
    # Query to extract relevant columns
    txt = '''SELECT
                positive,
                negative,
                achievements
            FROM user_feedback;'''
    query = text(txt)
    result = connection.execute(query)
    feedback_df = pd.DataFrame(result.all())

feedback_df

,positive,negative,achievements
0,198387.0,5135.0,0.0
1,5677.0,923.0,0.0
2,5179.0,566.0,0.0
3,1929.0,428.0,0.0
4,14333.0,718.0,0.0
...,...,...,...
85098,0.0,0.0,0.0
85099,0.0,0.0,0.0
85100,0.0,0.0,0.0
85101,0.0,0.0,0.0


In [21]:
# Fill NaN values with an empty string (if needed)
feedback_df = feedback_df.fillna('')

# Normalize numerical features using StandardScaler
numerical_columns = feedback_df.select_dtypes(include=['float64', 'int64']).columns
scaler = StandardScaler()
feedback_df[numerical_columns] = scaler.fit_transform(feedback_df[numerical_columns])

In [22]:
# Initialize PyCaret for regression
reg_setup = setup(
    data=feedback_df,
    target='positive',  # Assuming 'positive' is the target variable
    session_id=42
)

# Compare different regression models
reg_models = compare_models()

,Description,Value
0,Session id,42
1,Target,positive
2,Target type,Regression
3,Original data shape,"(85103, 3)"
4,Transformed data shape,"(85103, 3)"
5,Transformed train set shape,"(59572, 3)"
6,Transformed test set shape,"(25531, 3)"
7,Numeric features,2
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
knn,K Neighbors Regressor,0.0279,0.1630,0.3741,0.5465,0.0714,0.6833,0.1290
lightgbm,Light Gradient Boosting Machine,0.0285,0.1825,0.3906,0.4918,0.0701,0.7627,0.1290
rf,Random Forest Regressor,0.0291,0.1805,0.4017,0.4832,0.0729,0.7306,0.5390
gbr,Gradient Boosting Regressor,0.0274,0.1798,0.4012,0.4817,0.0665,0.7061,0.2620
et,Extra Trees Regressor,0.0291,0.1731,0.3975,0.4567,0.0744,0.7151,0.2650
dt,Decision Tree Regressor,0.0330,0.2352,0.4701,0.2613,0.0859,0.7431,0.0180
ada,AdaBoost Regressor,0.0848,0.2694,0.4833,0.1465,0.1222,2.3144,0.1250
huber,Huber Regressor,0.0255,0.4037,0.4876,0.1168,0.0656,0.4890,0.0650
en,Elastic Net,0.0661,0.3631,0.5723,-0.0002,0.1268,0.9619,0.0130
llar,Lasso Least Angle Regression,0.0661,0.3631,0.5723,-0.0002,0.1268,0.9619,0.0120


In [23]:
# Create a K Neighbors Regressor model
knn_model = create_model('knn')

# Evaluate the model
evaluate_model(knn_model)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0254,0.0977,0.3126,0.6523,0.0659,1.0692
1,0.0267,0.0743,0.2725,0.4086,0.0776,0.4549
2,0.0343,0.1796,0.4238,0.6684,0.0800,0.6071
3,0.0460,0.5466,0.7393,0.4200,0.0952,0.5074
4,0.0234,0.0451,0.2125,0.8488,0.0634,0.3181
5,0.0210,0.1125,0.3354,0.3643,0.0619,1.2880
6,0.0280,0.2948,0.5430,0.4729,0.0671,0.5414
7,0.0277,0.1325,0.3641,0.6460,0.0715,0.8656
8,0.0214,0.0586,0.2420,0.7225,0.0602,0.4809


interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [24]:
# # Split the data into features (X) and target variable (y)
# X = feedback_df[['positive', 'negative']]
# y = feedback_df['achievements']

# # Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Fit a linear regression model
# model = LinearRegression()
# model.fit(X_train, y_train)

# # Make predictions on the test set
# y_pred = model.predict(X_test)

# # Evaluate the model
# mse = mean_squared_error(y_test, y_pred)
# r2 = r2_score(y_test, y_pred)

# print(f'Mean Squared Error: {mse}')
# print(f'R-squared: {r2}')

# # Print the coefficients
# print(f'Intercept: {model.intercept_}, Coefficients: {model.coef_}')

# # Scatter plot of actual vs predicted values
# plt.scatter(X_test['positive'], y_test, label='Actual')
# plt.scatter(X_test['positive'], y_pred, label='Predicted')
# plt.title('Actual vs Predicted Achievements')
# plt.xlabel('Positive Reviews')
# plt.ylabel('Achievements')
# plt.legend()
# plt.show()

# # Regression plot
# sns.regplot(x=X_test['positive'], y=y_test, scatter_kws={'alpha': 0.3}, label='Actual')
# sns.regplot(x=X_test['positive'], y=y_pred, scatter_kws={'alpha': 0}, line_kws={'color': 'red'}, label='Regression Line')
# plt.title('Regression Plot for Actual vs Predicted Achievements')
# plt.xlabel('Positive Reviews')
# plt.ylabel('Achievements')
# plt.legend()
# plt.show()

# 5. Playtime Patterns:
- *Theory*: Games with longer median playtimes also have longer average playtimes.
- *Analysis*: Comparison of average_playtime_forever and median_playtime_forever by correlation analysis.

In [25]:
with engine.connect() as connection:
    # Query to extract relevant columns
    txt = '''SELECT
                appid,
                average_playtime_forever,
                median_playtime_forever
            FROM
                playtime;'''
    query = text(txt)
    result = connection.execute(query)
    playtime_df = pd.DataFrame(result.all())

playtime_df

,appid,average_playtime_forever,median_playtime_forever
0,10,10524.0,228.0
1,20,143.0,23.0
2,30,1397.0,27.0
3,40,945.0,10.0
4,50,214.0,66.0
...,...,...,...
85098,2760980,0.0,0.0
85099,2761170,0.0,0.0
85100,2763480,0.0,0.0
85101,2764930,0.0,0.0


In [26]:
# Fill NaN values with an empty string (if needed)
playtime_df = playtime_df.fillna('')

# Normalize numerical features using StandardScaler
numerical_columns = playtime_df.select_dtypes(include=['float64', 'int64']).columns.difference(['appid'])
scaler = StandardScaler()
playtime_df[numerical_columns] = scaler.fit_transform(playtime_df[numerical_columns])

In [27]:
# Initialize PyCaret for regression
reg_setup = setup(
    data=playtime_df,
    target='average_playtime_forever',  # Assuming 'average_playtime_forever' is the target variable
    session_id=42
)

# Compare different regression models
reg_models = compare_models()

,Description,Value
0,Session id,42
1,Target,average_playtime_forever
2,Target type,Regression
3,Original data shape,"(85103, 3)"
4,Transformed data shape,"(85103, 3)"
5,Transformed train set shape,"(59572, 3)"
6,Transformed test set shape,"(25531, 3)"
7,Numeric features,2
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lr,Linear Regression,0.0752,0.2574,0.4922,0.7300,0.1036,0.6814,0.0150
ridge,Ridge Regression,0.0752,0.2574,0.4921,0.7300,0.1036,0.6815,0.0120
lar,Least Angle Regression,0.0752,0.2574,0.4922,0.7300,0.1036,0.6814,0.0130
br,Bayesian Ridge,0.0752,0.2574,0.4921,0.7300,0.1036,0.6815,0.0180
rf,Random Forest Regressor,0.0518,0.2921,0.5191,0.6776,0.1063,0.5890,1.2130
et,Extra Trees Regressor,0.0550,0.3301,0.5547,0.6546,0.1121,0.6522,0.2260
gbr,Gradient Boosting Regressor,0.0484,0.3051,0.5313,0.6417,0.0919,0.5008,0.4400
dt,Decision Tree Regressor,0.0620,0.4290,0.6405,0.5049,0.1268,0.6747,0.0420
lightgbm,Light Gradient Boosting Machine,0.0549,0.5651,0.6944,0.5019,0.0977,0.4824,0.1240
en,Elastic Net,0.1248,0.6624,0.7487,0.4547,0.1325,1.1483,0.0130


In [28]:
# Create a Linear Regression model
lr_model = create_model('lr')

# Evaluate the model
evaluate_model(lr_model)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0710,0.1464,0.3826,0.6636,0.1025,0.6782
1,0.0785,0.1676,0.4094,0.6296,0.1202,0.5893
2,0.0875,0.5613,0.7492,0.8225,0.1097,0.7550
3,0.0768,0.4305,0.6562,0.8404,0.0969,0.6216
4,0.0732,0.1366,0.3697,0.7049,0.1049,0.7981
5,0.0723,0.2764,0.5257,0.5728,0.0998,0.7243
6,0.0719,0.2429,0.4929,0.8440,0.0898,0.5568
7,0.0712,0.1341,0.3662,0.7349,0.1033,0.5919
8,0.0782,0.3014,0.5490,0.7642,0.1078,0.8545


interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [29]:
# # Split the data into features (X) and target variable (y)
# X = playtime_df[['average_playtime_forever']]
# y = playtime_df['median_playtime_forever']

# # Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Fit a linear regression model
# model = LinearRegression()
# model.fit(X_train, y_train)

# # Make predictions on the test set
# y_pred = model.predict(X_test)

# # Evaluate the model
# mse = mean_squared_error(y_test, y_pred)
# r2 = r2_score(y_test, y_pred)

# print(f'Mean Squared Error: {mse}')
# print(f'R-squared: {r2}')

# # Print the coefficients
# print(f'Intercept: {model.intercept_}, Coefficient: {model.coef_}')

# # Regression plot
# plt.scatter(X_test, y_test, label='Actual')
# plt.scatter(X_test, y_pred, label='Predicted')
# plt.title('Actual vs Predicted Median Playtime')
# plt.xlabel('Average Playtime Forever')
# plt.ylabel('Median Playtime Forever')
# plt.legend()
# plt.show()

# # Regression plot
# sns.regplot(x=X_test, y=y_test, scatter_kws={'alpha': 0.3}, label='Actual')
# sns.regplot(x=X_test, y=y_pred, scatter_kws={'alpha': 0}, line_kws={'color': 'red'}, label='Regression Line')
# plt.title('Regression Plot for Actual vs Predicted Median Playtime')
# plt.xlabel('Average Playtime Forever')
# plt.ylabel('Median Playtime Forever')
# plt.legend()
# plt.show()

# 6. Price Influence
- *Theory*: User scores for games are typically higher when they are more expensive.
- *Analysis*: Examine average user scores for various price ranges of games.

In [30]:
with engine.connect() as connection:
    # Query to extract relevant columns
    txt = '''SELECT 
                game.appid, 
                game.price, 
                user_feedback.user_score
            FROM game
            JOIN user_feedback ON game.appid = user_feedback.appid;'''
    query = text(txt)
    result = connection.execute(query)
    price_df = pd.DataFrame(result.all())

price_df

,appid,price,user_score
0,10,9.99,0.0
1,20,4.99,0.0
2,30,4.99,0.0
3,40,4.99,0.0
4,50,4.99,0.0
...,...,...,...
85098,2760980,0.00,0.0
85099,2761170,0.00,0.0
85100,2763480,0.00,0.0
85101,2764930,0.00,0.0


In [31]:
# Fill NaN values with an empty string (if needed)
price_df = price_df.fillna('')

# Normalize numerical features using StandardScaler
numerical_columns = price_df.select_dtypes(include=['float64', 'int64']).columns.difference(['appid'])
scaler = StandardScaler()
price_df[['price', 'user_score']] = scaler.fit_transform(price_df[['price', 'user_score']])

In [32]:
# Initialize PyCaret for regression
reg_setup = setup(
    data=price_df,
    target='user_score',  # Assuming 'user_score' is the target variable
    session_id=42
)

# Compare different regression models
reg_models = compare_models()

,Description,Value
0,Session id,42
1,Target,user_score
2,Target type,Regression
3,Original data shape,"(85103, 3)"
4,Transformed data shape,"(85103, 3)"
5,Transformed train set shape,"(59572, 3)"
6,Transformed test set shape,"(25531, 3)"
7,Numeric features,2
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
huber,Huber Regressor,0.0095,0.3652,0.3745,-551619378508.9655,0.0334,0.0521,0.0940
dummy,Dummy Regressor,0.0453,1.0013,0.9334,-18246034417254.3008,0.0856,1.0184,0.0110
lasso,Lasso Regression,0.0462,1.0010,0.9339,-28687845302955.4805,0.0830,1.0601,0.0120
llar,Lasso Least Angle Regression,0.0462,1.0010,0.9339,-28687845302955.4883,0.0830,1.0601,0.0130
en,Elastic Net,0.0462,1.0010,0.9339,-28688636000700.4883,0.0830,1.0601,0.0130
omp,Orthogonal Matching Pursuit,0.0462,1.0010,0.9339,-28689426728389.8008,0.0830,1.0601,0.0120
br,Bayesian Ridge,0.0462,1.0011,0.9339,-28691219408631.6016,0.0830,1.0602,0.0150
ridge,Ridge Regression,0.0463,1.0011,0.9339,-29062248413368.4141,0.0830,1.0639,0.0130
lar,Least Angle Regression,0.0463,1.0011,0.9339,-29062262232541.6758,0.0830,1.0639,0.0120
lr,Linear Regression,0.0463,1.0011,0.9339,-29062262232541.7266,0.0830,1.0639,0.0160


In [33]:
# Create a Lasso Regression model
lasso_model = create_model('lasso')

# Evaluate the Lasso Regression model
evaluate_model(lasso_model)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0424,0.7659,0.8752,0.0004,0.0817,1.0626
1,0.0516,1.2527,1.1192,0.0004,0.0985,1.0191
2,0.0476,1.2199,1.1045,0.0002,0.0876,1.0485
3,0.0438,0.7868,0.8870,0.0001,0.0830,1.0886
4,0.0364,0.4109,0.6410,-0.0005,0.0661,1.1169
5,0.0535,1.4601,1.2083,0.0003,0.1002,1.0083
6,0.0477,1.2102,1.1001,0.0002,0.0881,1.0408
7,0.0641,1.9549,1.3982,0.0001,0.1189,0.9667
8,0.0482,0.9481,0.9737,0.0003,0.0943,1.0507


interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [34]:
# # Split the data into features (X) and target variable (y)
# X = price_df[['price']]
# y = price_df['user_score']

# # Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Fit a linear regression model
# model = LinearRegression()
# model.fit(X_train, y_train)

# # Make predictions on the test set
# y_pred = model.predict(X_test)

# # Evaluate the model
# mse = mean_squared_error(y_test, y_pred)
# r2 = r2_score(y_test, y_pred)

# print(f'Mean Squared Error: {mse}')
# print(f'R-squared: {r2}')

# # Print the coefficients
# print(f'Intercept: {model.intercept_}, Coefficient: {model.coef_}')

# # Regression plot
# plt.scatter(X_test, y_test, label='Actual')
# plt.scatter(X_test, y_pred, label='Predicted')
# plt.title('Actual vs Predicted User Scores by Price')
# plt.xlabel('Price')
# plt.ylabel('User Score')
# plt.legend()
# plt.show()

# # Regression plot
# sns.regplot(x=X_test, y=y_test, scatter_kws={'alpha': 0.3}, label='Actual')
# sns.regplot(x=X_test, y=y_pred, scatter_kws={'alpha': 0}, line_kws={'color': 'red'}, label='Regression Line')
# plt.title('Regression Plot for Actual vs Predicted User Scores by Price')
# plt.xlabel('Price')
# plt.ylabel('User Score')
# plt.legend()
# plt.show()

# 7. Categorization Impact on Popularity
- *Theory*: Owner estimates of games tend to be higher for those with more categories and genres.
- *Analysis*: Look at the connection between estimated owners and the number of categories/genres.

In [35]:
with engine.connect() as connection:
    # Query to extract relevant columns
    txt = '''SELECT 
                game.appid, 
                game.estimated_owners, 
                categorization.categories, 
                categorization.genres
            FROM game
            JOIN categorization ON game.appid = categorization.appid;'''
    query = text(txt)
    result = connection.execute(query)
    category_df = pd.DataFrame(result.all())

category_df

,appid,estimated_owners,categories,genres
0,10,15000000.0,"Multi-player,PvP,Online PvP,Shared/Split Scree...",Action
1,20,7500000.0,"Multi-player,PvP,Online PvP,Shared/Split Scree...",Action
2,30,7500000.0,"Multi-player,Valve Anti-Cheat enabled",Action
3,40,7500000.0,"Multi-player,PvP,Online PvP,Shared/Split Scree...",Action
4,50,7500000.0,"Single-player,Multi-player,Valve Anti-Cheat en...",Action
...,...,...,...,...
85098,2760980,0.0,unknown,unknown
85099,2761170,0.0,unknown,unknown
85100,2763480,0.0,unknown,unknown
85101,2764930,0.0,unknown,unknown


In [36]:
# # Download NLTK stopwords data
# nltk.download('stopwords')

# Columns to convert to lists
columns_to_convert = [ 'categories', 'genres']

# Fill NaN values with an empty string
category_df[columns_to_convert] = category_df[columns_to_convert].fillna('')

# Normalize numerical features using StandardScaler
numerical_columns = category_df.select_dtypes(include=['float64', 'int64']).columns.difference(['appid'])
scaler = StandardScaler()
category_df[numerical_columns] = scaler.fit_transform(category_df[numerical_columns])

# One-hot encode categorical features
categorical_columns = category_df.select_dtypes(include=['bool']).columns.difference(['appid'])
category_df = pd.get_dummies(category_df, columns=categorical_columns, drop_first=True)

# # Use CountVectorizer to convert text columns to bag-of-words representation
# count_vectorizer = CountVectorizer(stop_words=stopwords.words('english'))  # Use English stopwords
# for column in columns_to_convert:
#     column_bow = count_vectorizer.fit_transform(category_df[column])
#     category_df = pd.concat([category_df, pd.DataFrame(column_bow.toarray(), columns=count_vectorizer.get_feature_names_out([column]))], axis=1)

category_df

,appid,estimated_owners,categories,genres
0,10,14.511447,"Multi-player,PvP,Online PvP,Shared/Split Scree...",Action
1,20,7.213937,"Multi-player,PvP,Online PvP,Shared/Split Scree...",Action
2,30,7.213937,"Multi-player,Valve Anti-Cheat enabled",Action
3,40,7.213937,"Multi-player,PvP,Online PvP,Shared/Split Scree...",Action
4,50,7.213937,"Single-player,Multi-player,Valve Anti-Cheat en...",Action
...,...,...,...,...
85098,2760980,-0.083573,unknown,unknown
85099,2761170,-0.083573,unknown,unknown
85100,2763480,-0.083573,unknown,unknown
85101,2764930,-0.083573,unknown,unknown


In [37]:
# Initialize PyCaret setup
reg_setup = setup(
    data=category_df,
    target='estimated_owners',
    train_size=0.8,
    session_id=42,
    numeric_features=[],  # Specify numeric features if needed
    categorical_features=['categories', 'genres'],  # Specify categorical features
    ignore_features=['appid'],  # Specify features to ignore
    fold=5,  # Specify the number of folds for cross-validation
    # normalize=True,
    # transformation=True,
    # transform_target=True
)

# Compare different regression models
reg_models = compare_models()

,Description,Value
0,Session id,42
1,Target,estimated_owners
2,Target type,Regression
3,Original data shape,"(85103, 4)"
4,Transformed data shape,"(85103, 3)"
5,Transformed train set shape,"(68082, 3)"
6,Transformed test set shape,"(17021, 3)"
7,Ignore features,1
8,Categorical features,2
9,Preprocess,True


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
huber,Huber Regressor,0.0792,1.1310,1.0203,0.0003,0.1645,0.4334,0.0880
llar,Lasso Least Angle Regression,0.1316,1.1308,1.0204,-0.0002,0.1833,1.0258,0.0440
lasso,Lasso Regression,0.1316,1.1308,1.0204,-0.0002,0.1833,1.0258,0.0860
dummy,Dummy Regressor,0.1316,1.1308,1.0204,-0.0002,0.1833,1.0258,0.0400
en,Elastic Net,0.1316,1.1308,1.0204,-0.0002,0.1833,1.0258,0.0640
gbr,Gradient Boosting Regressor,0.0953,1.1559,1.0324,-0.0270,0.1689,0.8337,0.4120
knn,K Neighbors Regressor,0.0970,1.1692,1.0387,-0.0407,0.1721,0.9092,0.0740
rf,Random Forest Regressor,0.0965,1.1700,1.0400,-0.0462,0.1717,0.8892,0.7000
ada,AdaBoost Regressor,0.1506,1.1869,1.0478,-0.0615,0.1783,1.5026,0.1120
et,Extra Trees Regressor,0.0968,1.1799,1.0458,-0.0624,0.1727,0.9031,0.3720


In [38]:
# Create a Huber Regressor model
huber_model = create_model('huber')

# Evaluate the Huber Regressor model
evaluate_model(huber_model)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0843,2.3891,1.5457,-0.0007,0.1646,0.4424
1,0.0768,0.7957,0.8920,-0.0003,0.1643,0.4316
2,0.0928,1.2090,1.0995,-0.0011,0.1835,0.4474
3,0.0654,0.4162,0.6452,0.0036,0.1489,0.4174
4,0.0765,0.8452,0.9193,-0.0001,0.1611,0.4282
Mean,0.0792,1.1310,1.0203,0.0003,0.1645,0.4334
Std,0.0091,0.6773,0.2999,0.0017,0.0111,0.0106


interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…